In [1]:
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
import nltk
import json
import math
from nltk.util import pr
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans
from sklearn import svm
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from nltk.tokenize import word_tokenize
import os
from joblib import dump, load
Size = 0


In [35]:

# functions

def make_idf(lexicon):
    idf = dict()
    global Size
    N = Size
    for term,docs in lexicon.items():
        df = len(docs)
        temp = math.log(N / df)
        idf[term] = temp
    return idf

def cleaner(uncleaned):
    porter = PorterStemmer()
    lemma = WordNetLemmatizer()
    uncleaned = uncleaned.lower()
    uncleaned = uncleaned.strip()
    uncleaned = uncleaned.translate(
        {ord(i): None for i in '!\\@#-_:$%^&*();.,?/1”2’3“4‘567890\'\"'})
    for i in uncleaned:
        t = ord(i)
        if t < 97 or t > 122:
            uncleaned = uncleaned.replace(i, "")
    uncleaned = porter.stem(uncleaned)
    uncleaned = lemma.lemmatize(uncleaned, pos='v')
    return uncleaned

def Read_Data_From_File():
    Tokens = []
    alpha = ''
    Raw_Data = dict()
    X_headline = []
    Y_Sarcastic = []
    i = 0
    count = 0
    lexicon = {}

    stop_word = []
    with open("Stopword-List.txt", 'r') as stop:
        for line in stop:
            temp = line.strip()
            stop_word.append(temp)

    for lines in open('Sarcasm_Headlines_Dataset.json', 'r'):
        count += 1
        Raw_Data[i] = json.loads(lines)
        X_headline.append(Raw_Data[i]["headline"])
        Y_Sarcastic.append(Raw_Data[i]["is_sarcastic"])

        #tokenize here
        words = word_tokenize(X_headline[i])
        curr = str(i)
        for word in words:
            if len(word) >= 3:
                if word in stop_word:
                    continue
                taggs = nltk.tag.pos_tag([word])
                if(taggs[0][1] != 'NNP' and taggs[0][1] != 'FW' and taggs[0][1] != 'PRP'):
                    word = cleaner(word)
                    if len(word) >= 3:                 
                        # Tokens.append(word)

                        if word in lexicon:
                            if curr in lexicon[word]:
                                lexicon[word][i] += 1
                            else:
                                lexicon[word][i] = 1
                        else:
                            lexicon[word] = {}
                            if curr in lexicon[word]:
                                lexicon[word][i] += 1
                            else:
                                lexicon[word][i] = 1
        i = i+1
        # print(lexicon)
    global Size 
    Size = count
    return lexicon

def tf_idf_lexicon(lexicon, idf):
    tf_idf = {}
    for term, docs in lexicon.items():
        tf_idf[term] = {}
        for docNo, tf in docs.items():
            tf_idf[term][docNo] = tf * idf[term]

    return tf_idf

def applyPCA(df):
    print("In function")
    scaler = StandardScaler()
    #scaler=MinMaxScaler()
    print("Standard scaler defined")
    scaler.fit(df)
    print("scaler Fit defined")
    scaled_data = scaler.transform(df)
    print("scaler trasnform")

    pca = PCA(n_components=900)
    pca.fit(scaled_data)
    x_pca = pca.transform(scaled_data)
    print("Data Variance: ", sum(pca.explained_variance_ratio_)*100)

    return x_pca, scaled_data


def sort(tfidf):
    size = 26709
    list1 = []
    check = False
    for i in range(size):
        list1.append(i+1)
    # print(list1)
    
    for word,doc in tfidf.items():
        print(word)
        for i in list1:
            if i not in doc.keys():
                tfidf[word][i] = 0
    return tfidf

def make_idf1(Tokens):
    list1 = []
    idf = dict()
    global Size
    N = Size
    # for term,docs in lexicon.items():
    #     df = len(docs)
    #     temp = math.log(N / df)
    #     idf[term] = temp
    # return idf
    for i in Tokens:
        # print(i[0],i[1])        
        count =  i[0]
        while i[0]+1 == count:
             list1.append(Tokens[i[1]])
    print(list1)


In [23]:
lexicon = Read_Data_From_File()
print("lexicon")
idf = make_idf(lexicon)
print("Idf made")
tf_idf = tf_idf_lexicon(lexicon,idf)
print("tf_Idf made")
# tf_idf = sort(tf_idf)
with open("TFIDF_Dictionary.json", "w") as f: # Writing the index to JSON File.
   f.write(json.dumps(tf_idf, sort_keys=False, indent=4)) 

lexicon
Idf made
tf_Idf made


In [24]:
with open("IDF_Dictionary.json", "w") as f: # Writing the index to JSON File.
   f.write(json.dumps(idf, sort_keys=False, indent=4)) 

In [25]:
len(idf)

18616

In [39]:
x_pca

(26709, 18616)
18616
Data Processing Completed........

Applying PCA.....................
In function
Standard scaler defined
scaler Fit defined
scaler trasnform
Data Variance:  22.381134315944273
[[-0.05438118 -0.0317108   0.0074228  ... -0.26486439  0.04284025
  -1.01061468]
 [-0.04007727 -0.01453717 -0.03777587 ...  0.11131384  0.08171687
  -0.34716208]
 [-0.08269532 -0.04001263 -0.02362679 ...  4.43565155 -2.92950651
  -4.68224005]
 ...
 [-0.03373501 -0.01018237 -0.03598306 ... -0.24622564 -0.08508081
  -0.0480576 ]
 [-0.06268075 -0.02089338  0.00949616 ... -2.27411292  1.34326509
  -1.02793123]
 [-0.03056064 -0.00666135 -0.03710731 ... -0.12211445  0.19533831
  -0.04966335]] [[ 1.59176452e+01  0.00000000e+00  2.49026010e+01 ...  0.00000000e+00
  -6.11898398e-03  0.00000000e+00]
 [-6.28233629e-02  0.00000000e+00 -4.01564480e-02 ...  0.00000000e+00
  -6.11898398e-03  0.00000000e+00]
 [-6.28233629e-02  0.00000000e+00 -4.01564480e-02 ...  0.00000000e+00
  -6.11898398e-03  0.00000000e+

In [2]:

with open('TFIDF_Dictionary.json') as json_file:
    Dict = json.load(json_file)
# Dict = sort(Dict)
# print(Dict)
# with open("TFIDF_Dictionary_new.json", "w") as f:
    # f.write(json.dumps(Dict, sort_keys=False, indent=4))

#Create Panda DataFrame
#    print("Hello")
df = pd.DataFrame(Dict.items())
vocab = list(Dict.keys())
vocab_len = len(vocab)
word_to_inx = {}
for word in vocab:
    word_to_inx[word] = vocab.index(word)
i = 0
dataset = np.empty((26709, vocab_len))
for line in open('Sarcasm_Headlines_Dataset.json', 'r'):
    data = json.loads(line)
    str_i = str(i)
    docVector = np.zeros(vocab_len)
    for word in data["headline"].split(" "):
        if word in word_to_inx and str_i in Dict[word]:
            wordPos = word_to_inx[word]
            docVector[wordPos] = Dict[word][str_i]
            dataset[i] = docVector
    i += 1

print(dataset.shape)

# df = np.transpose(df)
# print(df[0])



(26709, 18616)


In [30]:
dataset /= np.max(dataset)

In [31]:
dataset

array([[0.5434051 , 0.        , 0.60163443, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 1.        ,
        0.        ]])

In [3]:
print(vocab_len)

print("Data Processing Completed........")
print("")
print("Applying PCA.....................")

# x_pca, scaled_data = applyPCA(dataset)
print("In function")
scaler = StandardScaler()
#scaler=MinMaxScaler()
print("Standard scaler defined")
scaler.fit(dataset)
print("scaler Fit defined")
scaled_data = scaler.transform(dataset)
print("scaler trasnform")

# pca = PCA(n_components=900)
# pca.fit(scaled_data)
# x_pca = pca.transform(scaled_data)
# print("Data Variance: ", sum(pca.explained_variance_ratio_)*100)

# print(x_pca,scaled_data)
# print("")
# print("PCA Applied......................")
# print("")
# print("Traning Model....................")

18616
Data Processing Completed........

Applying PCA.....................
In function
Standard scaler defined
scaler Fit defined
scaler trasnform


In [4]:
# dump(pca, 'pca.joblib')

['pca.joblib']

In [5]:
# dump(scaler, 'scaler.joblib')

['scaler.joblib']

In [4]:
X=[]
Y=[]

Raw_Data = dict()
X_headline = []
Y_Sarcastic = []
i = 0
count = 0

for lines in open('Sarcasm_Headlines_Dataset.json', 'r'):
    count += 1
    Raw_Data[i] = json.loads(lines)
    X_headline.append(Raw_Data[i]["headline"])
    Y_Sarcastic.append(Raw_Data[i]["is_sarcastic"])

size=len(X_headline)


In [39]:
len(Y_Sarcastic)

26709

In [5]:
# from sklearn.feature_selection import SelectKBest, mutual_info_classif
# X_new = SelectKBest(mutual_info_classif, k=4000).fit_transform(dataset, Y_Sarcastic)
from sklearn.feature_selection import SelectKBest, chi2
X_new = SelectKBest(chi2, k=4000).fit_transform(dataset, Y_Sarcastic)

In [6]:
#x_pca.to_csv('pData.csv')
#x_pca=pd.read_csv('pData')

#BreakData
X_train, X_test, y_train, y_test = train_test_split(X_new, Y_Sarcastic,test_size=0.30,random_state=48)

print("Applying SVM.....................")





Applying SVM.....................


In [7]:
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=18)
neigh.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=18)

In [8]:
neigh.score(X_test, y_test)

0.6651691002121553

In [9]:
#Apply SVM
model = svm.SVC(random_state=20)
print("Model Trained....................")
print("")
print("Applying Model...................")



Model Trained....................

Applying Model...................


In [10]:
model.fit(X_train,y_train)


In [46]:

# dump(model, 'model_70acc.joblib')

['model_70acc.joblib']

In [47]:
# new_model = load('model_70acc.joblib') 

In [1]:
acc=model.score(X_test,y_test)

print("")
print("Accuracy= ",acc*100,"%")

NameError: name 'model' is not defined

In [13]:
acc=model.score(X_train,y_train)

print("")
print("Accuracy= ",acc*100,"%")


Accuracy=  77.21720409978003 %


In [15]:
# X_test

In [49]:
model.predict([X_test[0]])

array([0])

In [30]:


with open('IDF_Dictionary.json') as json_file:
    idf2 = json.load(json_file)
vocab_len = len(idf2)
query_dataset = np.empty((1, vocab_len))
j = 0
for i in query_dataset:
    for x in i:
        # print(x)
        query_dataset[0][j] = 0
        j+=1

In [95]:
query = "after careful consideration, bush recommends oil drilling"
tf_query = idf2.copy()
for key in tf_query.keys():
    tf_query[key] = 0.0
words = word_tokenize(query)

stop_word = []
with open("Stopword-List.txt", 'r') as stop:
    for line in stop:
        temp = line.strip()
        stop_word.append(temp)

for word in words:
    if len(word) >= 3:
        if word in stop_word:
            continue
        taggs = nltk.tag.pos_tag([word])
        if(taggs[0][1] != 'NNP' and taggs[0][1] != 'FW' and taggs[0][1] != 'PRP'):
            word = cleaner(word)
            if len(word) >= 3:                 
                # Tokens.append(word)
                
                if word in tf_query.keys():
                    if tf_query[word] == 0:
                        tf_query[word] = idf2[word]
                    else:
                        tf_query += idf2[word]
# print(lexicon)

In [96]:
j = 0
for i in tf_query.values():
    query_dataset[0][j] = i
    j+=1

In [97]:
query_dataset.shape

(1, 18616)

In [98]:
pca_q = load('pca.joblib') 

In [99]:
# query_dataset.shape

print("Data Processing Completed........")
print("")
print("Applying PCA.....................")

# test_pca, scaled_data = applyPCA(query_dataset)
print("In function")
scaler2 = StandardScaler()
#scaler=MinMaxScaler()
print("Standard scaler defined")
scaler2.fit(query_dataset)
print("scaler Fit defined")
scaled_data_query = scaler2.transform(query_dataset)
print("scaler trasnform")

# pca_q = PCA(n_components=3000)
# pca_q.fit(scaled_data_query)
test_pca = pca_q.transform(scaled_data_query)
print("Data Variance: ", sum(pca_q.explained_variance_ratio_)*100)

print(test_pca,scaled_data)
print("")
print("PCA Applied......................")
print("")
print("Traning Model....................")

Data Processing Completed........

Applying PCA.....................
In function
Standard scaler defined
scaler Fit defined
scaler trasnform
Data Variance:  22.381134315944273
[[ 1.51939087e-15  2.59625184e-16  3.10229874e-15  3.62345360e-16
   1.45243569e-16 -5.17010932e-16 -1.75750204e-15 -9.60684602e-16
   3.07710810e-16  2.84391934e-16 -1.38842103e-16  1.27785708e-15
   2.54894132e-15  2.84137885e-16 -1.57731741e-15 -2.14532250e-15
   1.21496553e-15 -5.07656539e-16  1.71718332e-15 -1.76198725e-15
   6.75226504e-16  8.26895453e-16 -5.32388192e-16 -1.68820230e-15
  -3.99993796e-16  1.00120198e-16  8.80896592e-16 -1.28346174e-15
   9.04670440e-16 -4.14883692e-16  1.64495553e-16  1.45658437e-15
   2.35583178e-15  2.49476110e-15  1.02622506e-15  3.41311465e-16
   5.64902206e-16 -2.57179624e-16  3.95657220e-16  2.71545153e-15
   1.57584008e-15 -8.81835079e-16  8.83505040e-16  1.22598978e-15
  -7.65177298e-16  1.42513730e-15  9.26072975e-17  6.55773366e-16
   2.12582524e-15  6.47243680e-1

In [100]:
test_pca.shape

(1, 900)

In [101]:
new_model = load('model_70acc.joblib') 

In [102]:
new_model.predict(test_pca)

array([0])

In [10]:
# import sklearn.externals.joblib as extjoblib
# import joblib
# joblib.dump(clf, 'filename.pkl')

In [10]:
##Test
# pca=PCA(n_components=2)
# pca.fit(scaled_data)
# x_pca=pca.transform(scaled_data)

# plt.figure(figsize=(20,12))
# plt.scatter(x_pca[:,0],x_pca[:,1])
# plt.xlabel('First principle component')
# plt.ylabel('Second principle component')